## **Setup**

In [ ]:
from lib.submitter import *

jwt = ''      # User-specified field.
address = ''  # User-specified field.
pk = ''       # User-specified field.
submitter = Submitter(jwt, address, pk)

## View wallet address information.

In [ ]:
print('MATIC balance for {}: {}'.format(submitter.address, submitter.get_matic_balance()))
print('MUSA balance for {}: {}'.format(submitter.address, submitter.get_musa_balance()))
print('Current stake for {}: {}'.format(submitter.address, submitter.get_stake()))

## **Retrieve Dataset and run models.**
### Please follow the instructions and example found [here](https://github.com/rocketcapital-ai/competition_quickstart_dataloader).

### When completed, please place final prediction csv file in the `file_to_submit` folder.

## **Set stake.** (optional)
### Skip if no changes need to be made.

In [ ]:
set_stake_amount = 100.00  # User-specified field.
transaction_success = submitter.set_stake(set_stake_amount)
assert transaction_success, 'Setting stake failed.'

## **Submit predictions.**
Note that the same `submit_prediction` method should be used for re-submitting predictions.

In [ ]:
submission_file_name = 'my_submission_file.csv'  # User-specified field.
transaction_success = submitter.submit_prediction(submission_file_name)
assert transaction_success, 'Submission failed.'

### **Retrieve and double-check predictions.** (optional)
This section retrieves your submitted files, decrypts them, and compares them to the original file in `file_to_submit`.

If the verification fails, please wait a few minutes and perform the verification again. If the problem persists, please re-submit your predictions.

In [ ]:
verification_success = submitter.download_and_check(submission_file_name)
assert verification_success, 'Submission verification failed.'
print('Files are identical. Verification check passed.')